In [32]:
# 使用连续分类器进行词性标注

In [33]:
import nltk

In [34]:
def pos_features(sentence, i, history):
    '''
    特征构造生成器
    入参：sentence 句子列表
        i 句子中取的词所在的index值
        history sentence对应的tag标签列表
    '''
    features = {'suffix(1)':sentence[i][-1:],
               'suffix(2)':sentence[i][-2:],
               'suffix(3)':sentence[i][-3:]}
    if i == 0:
        features['prev-word'] = '<START>'
        features['prev-tag'] = '<START>'
    else:
        features['prev-word'] = sentence[i-1]
        features['prev-tag'] = history[i-1]
    return features

In [53]:
class ConsecutivePosTagger(nltk.TaggerI):
    def __init__(self, train_sents):
        train_set = []
        for tagged_sent in train_sents:
            untagged_sent = nltk.tag.untag(tagged_sent)
            history = []
            for i,(word,tag) in enumerate(tagged_sent):
                featureset = pos_features(untagged_sent, i ,history)
                train_set.append((featureset,tag))
                history.append(tag)
        self.classifier = nltk.NaiveBayesClassifier.train(train_set)
    
    def tag(self, sentence):
        history = []
        for i,word in enumerate(sentence):
            featureset = pos_features(sentence, i, history)
            tag = self.classifier.classify(featureset)
            history.append(tag)
        return zip(sentence, history)  # 生成[(s1,h1),(s2,h2),...]
    

In [54]:
tagged_sents = brown.tagged_sents(categories='news')

In [55]:
size = int(len(tagged_sents)*0.1)

In [56]:
train_sents,test_sents = tagged_sents[size:], tagged_sents[:size]

In [57]:
tagger = ConsecutivePosTagger(train_sents)

In [58]:
# 准确度
tagger.evaluate(test_sents)

0.7980528511821975